In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from utils.transformations import ExtendedTransformation, SimpleTransformation
from utils.filters import SimpleFilter
from sklearn.model_selection import train_test_split
import optuna
import pandas as pd
import numpy as np

In [ ]:
df_train = pd.read_csv("train_data/preprocessed/train_data.csv")
X_train, y_train = df_train.drop(columns=['Price']), df_train[['Price']]
preprocessor = ExtendedTransformation()
filter = SimpleFilter()
preprocessor.fit(X_train, y_train)
X_processed, y_processed = preprocessor.transform(X_train, y_train)
filter.fit(X_processed, y_processed)
X_filtered, y_filtered = filter.transform(X_processed, y_processed)

In [ ]:
df_test = pd.read_csv("train_data/preprocessed/test_data.csv")
X_test, y_test = df_test.drop(columns=['Price']), df_test[['Price']]
X_test_proccesed, y_test_proccessed = preprocessor.transform(X_test, y_test)
X_test_filtered, y_test_filtered = filter.transform(X_test_proccesed, y_test_proccessed)

In [ ]:
def create_model(trial):
    n_layers = trial.suggest_int('n_layers', 1, 6)
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten())
    for layer in range(n_layers):
        n_neuros = trial.suggest_categorical(f'neurons_of_layer{layer}', [32,64,128,256,512])
        model.add(tf.keras.layers.Dense(n_neuros, activation='relu'))
        dropout_ratio = trial.suggest_float(f'dropout_of_layer{layer}', 0.0, 0.35)
        model.add(tf.keras.layers.Dropout(dropout_ratio))
    model.add(tf.keras.layers.Dense(1))
    return model

In [ ]:
def objective(trial):
    X,y = X_filtered, y_filtered
    X_train, X_val, y_train, y_val = train_test_split(X,y, test_size=0.3, random_state=42)
    learning_rate = trial.suggest_float("learnin_rate", 0.0001, 0.1, log=True)
    model = create_model(trial)
    optimizer = tf.keras.optimizers.Adam(learning_rate)
    model.compile(loss='mse', metrics=['mae'], optimizer=optimizer)
    early_stopping_callback = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',  # Monitoriza la pérdida en el conjunto de validación
        patience=10,         # Número de épocas sin mejora antes de detener el entrenamient
        verbose=1,           # Muestra mensajes cuando se detiene el entrenamiento
        restore_best_weights=True  # Restaura los pesos del modelo de la mejor época encontrada
        )
    batch_size_param = trial.suggest_categorical('batch_size',[8,16,32,124])
    history = model.fit(X_train, y_train,
                        epochs=200, 
                        batch_size=batch_size_param,
                        validation_data=(X_val, y_val),
                        callbacks=[early_stopping_callback], verbose=0)
    
    results = model.evaluate(X_val, y_val, verbose=0)
    return results[0], model

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)
print(study.best_trial)